In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC,LinearSVC
import numpy as np
import seaborn as sns
from sklearn.metrics import make_scorer, f1_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score, roc_auc_score
from sklearn.metrics import classification_report, precision_recall_curve
from sklearn.model_selection import StratifiedKFold, KFold,RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import train_test_split

## Support Vector Machines (lineal y polinomico)

In [2]:
#No recuerdo como utilizar el df directamente desde visual studio asi que lo baje a un csv y lo subi de vuelta
df = pd.read_csv("train_data_final.csv")

outliers = pd.read_csv("outliers.csv")

df.shape

df.columns

Index(['Unnamed: 0', 'customer_ID', 'S_2', 'P_2', 'D_39', 'B_1', 'B_2', 'R_1',
       'S_3', 'D_41',
       ...
       'D_137', 'D_138', 'D_139', 'D_140', 'D_141', 'D_142', 'D_143', 'D_144',
       'D_145', 'target'],
      dtype='object', length=192)

In [3]:
outliers.merge(df,left_on='Unnamed: 0.1',right_on='Unnamed: 0')

,Unnamed: 0_x,Unnamed: 0.1,P_2_x,D_39_x,B_1_x,B_2_x,R_1_x,S_3_x,D_41_x,B_3_x,...,D_137,D_138,D_139_y,D_140_y,D_141_y,D_142,D_143_y,D_144_y,D_145_y,target_y
0,27,1281887,-0.082594,1.036996,0.123351,0.001927,1.005303,0.344413,0.372042,0.464230,...,NaN,NaN,0.001714,0.003163,0.002862,NaN,0.007378,0.009312,0.003965,1
1,34,4490661,0.832462,0.003122,0.053585,0.048048,0.252569,0.327033,0.291467,0.122791,...,NaN,NaN,1.007146,0.006433,0.864593,0.025416,1.009928,0.007565,0.099242,0
2,36,300683,0.213851,0.595342,0.331941,0.021597,0.759581,0.325903,0.003911,0.167722,...,1.002562,0.006102,1.001172,0.009062,0.947247,0.388294,1.005335,0.676655,0.093158,1
3,56,5284963,0.465945,0.002313,0.164618,0.002365,1.506865,0.477885,0.495657,0.132093,...,NaN,NaN,1.007453,0.005539,0.977547,0.536497,1.009631,0.007232,0.092750,1
4,82,642684,0.174441,1.597862,0.477614,0.073237,1.004416,0.173714,0.349823,0.381071,...,NaN,NaN,0.002016,0.005840,0.002964,NaN,0.000630,0.006430,0.002930,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13824,276512,5402429,-0.315471,0.388769,0.382666,0.029319,1.256538,1.635705,0.008130,0.086147,...,NaN,NaN,0.004617,0.000029,0.004765,NaN,0.003295,0.007983,0.000623,1
13825,276516,89037,0.069906,0.000666,0.222584,0.011646,1.007391,0.264498,0.619245,0.561674,...,NaN,NaN,1.003025,0.002155,0.950759,0.406878,1.006947,0.214255,0.187322,1
13826,276539,518271,0.310032,0.447816,0.602612,0.018587,0.506903,0.353268,0.008056,0.129600,...,0.009680,0.500699,0.003789,0.005098,0.007009,NaN,0.002777,0.004402,0.004784,1
13827,276563,3651786,0.252494,0.329432,0.165080,0.028412,0.507827,0.164967,0.006664,0.486884,...,NaN,NaN,1.002873,0.002082,0.985065,0.597239,1.004762,0.000193,0.098830,1


In [4]:
df = df[~df['Unnamed: 0'].isin(outliers['Unnamed: 0.1'])]

In [5]:
categorical_variables = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68'] 

df.drop(categorical_variables,axis=1,inplace=True)

payment_variables = []
spending_variables = []
delinquency_variables = []
balance_variables = []
risk_variables = []
for column in df.columns:
    if 'P' in column:
        payment_variables.append(column)
    if 'S' in column:
        spending_variables.append(column)
    if 'D' in column and 'ID' not in column:
        delinquency_variables.append(column)
    if 'B' in column:
        balance_variables.append(column)
    if 'R' in column:
        risk_variables.append(column)

In [6]:
df.drop('customer_ID',axis=1,inplace=True)

In [7]:
df = df.drop('S_2',axis=1)\
.drop('D_42',axis=1) \
.drop('D_49',axis=1) \
.drop('D_53',axis=1) \
.drop('D_73',axis=1) \
.drop('D_76',axis=1) \
.drop('R_9',axis=1)  \
.drop('D_82',axis=1) \
.drop('B_29',axis=1) \
.drop('D_87',axis=1) \
.drop('D_88',axis=1) \
.drop('D_106',axis=1)\
.drop('R_26',axis=1) \
.drop('D_108',axis=1)\
.drop('D_110',axis=1)\
.drop('D_111',axis=1)\
.drop('B_39',axis=1) \
.drop('B_42',axis=1) \
.drop('D_132',axis=1)\
.drop('D_134',axis=1)\
.drop('D_135',axis=1)\
.drop('D_136',axis=1)\
.drop('D_137',axis=1)\
.drop('D_138',axis=1)\
.drop('D_142',axis=1)\


delinquency_variables_redux = delinquency_variables.copy()



delinquency_variables_redux.remove('D_42') 
delinquency_variables_redux.remove('D_49') 
delinquency_variables_redux.remove('D_53') 
delinquency_variables_redux.remove('D_73') 
delinquency_variables_redux.remove('D_76') 
delinquency_variables_redux.remove('D_82') 
delinquency_variables_redux.remove('D_87') 
delinquency_variables_redux.remove('D_88') 
delinquency_variables_redux.remove('D_106')
delinquency_variables_redux.remove('D_108')
delinquency_variables_redux.remove('D_110')
delinquency_variables_redux.remove('D_111')
delinquency_variables_redux.remove('D_132')
delinquency_variables_redux.remove('D_134')
delinquency_variables_redux.remove('D_135')
delinquency_variables_redux.remove('D_136')
delinquency_variables_redux.remove('D_137')
delinquency_variables_redux.remove('D_138')
delinquency_variables_redux.remove('D_142')


df.loc[:,delinquency_variables_redux].fillna(0,inplace=True)

for col in df.columns:
    #print(col)
    df[col].fillna(df[col].mean(),inplace=True)

#### Armado del train_test

In [8]:
train_variables = []

for v in payment_variables:
    if v in df.columns:
        train_variables.append(v)
for v in balance_variables:
    if v in df.columns:        
        train_variables.append(v)
for v in risk_variables:
    if v in df.columns:        
        train_variables.append(v)     
for v in spending_variables:
    if v in df.columns:        
        train_variables.append(v)       
#for v in delinquency_variables_redux:
#    if v in df.columns:        
#        train_variables.append(v)        
        
        

#X = df.iloc[0:20000,:][train_variables]
X = df[train_variables]
        
#X = df.drop(['target'],axis=1)
#X = X.drop(delinquency_variables_redux,axis=1)
#X = df[payment_variables]
#y = df.iloc[0:20000,:].target
y = df.target
features = X.columns





X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
X_train_norm=scaler.fit_transform(X_train)
X_test_norm=scaler.fit_transform(X_test)

X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 262743 entries, 0 to 276571
Data columns (total 85 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   P_2     262743 non-null  float64
 1   P_3     262743 non-null  float64
 2   P_4     262743 non-null  float64
 3   B_1     262743 non-null  float64
 4   B_2     262743 non-null  float64
 5   B_3     262743 non-null  float64
 6   B_4     262743 non-null  float64
 7   B_5     262743 non-null  float64
 8   B_6     262743 non-null  float64
 9   B_7     262743 non-null  float64
 10  B_8     262743 non-null  float64
 11  B_9     262743 non-null  float64
 12  B_10    262743 non-null  float64
 13  B_11    262743 non-null  float64
 14  B_12    262743 non-null  float64
 15  B_13    262743 non-null  float64
 16  B_14    262743 non-null  float64
 17  B_15    262743 non-null  float64
 18  B_16    262743 non-null  float64
 19  B_17    262743 non-null  float64
 20  B_18    262743 non-null  float64
 21  B_19    26

### Hiperparametros lineales y polinomicos para el svm

In [9]:
#Seteo hyperparametros lineales y polinomicas
param_dist_polinomic = {"kernel" : ['poly'] ,
            "C" : [1e+0,1e+1,1e+2,1e+3,1e+4,1e+5,1e+6,1e+7,1e+8,1e+9],
            "degree": [1e-4,1e-3,1e-2,1e-1,1e+0,1e+1,1e+2,1e+3,1e+4],
            "gamma": [1e-4,1e-3,1e-2,1e-1,1e+0,1e+1,1e+2,1e+3,1e+4],
            "coef0": [1e-4,1e-3,1e-2,1e-1,1e+0,1e+1,1e+2,1e+3,1e+4]
              }

#param_dist_linear = {"kernel":['linear'],
#             "C" : [1e+0,1e+1,1e+2,1e+3,1e+4,1e+5,1e+6,1e+7,1e+8,1e+9] }

param_dist_linear = {
             "C" : [1e+0,1e+1,1e+2,1e+3,1e+4,1e+5,1e+6,1e+7,1e+8,1e+9] }

In [10]:
random_search_polynomic_svc = RandomizedSearchCV(SVC(random_state=0),
                                       param_distributions = param_dist_polinomic,
                                       n_iter = 2,
                                       scoring = 'roc_auc',
                                       cv = 5,
                                       n_jobs = 2)



#random_search_linear_svc = RandomizedSearchCV(SVC(random_state=0,tol=1e-3,max_iter=20000),
#                                       param_distributions = param_dist_linear,
#                                       n_iter = 2,
#                                       scoring = 'roc_auc',
#                                       cv = 5,
#                                       n_jobs = 2)

#parameters = {'kernel':('linear','rbf','poly'),'C':[1,1e+7],'degree':[3,8],'tol':[1,1e-7],'max_iter':[1,100000]}
parameters = {'kernel':('rbf','poly'),'C':[1,1e+7],'degree':[8],'tol':[1e-7],'max_iter':[50000,100000]}

svc_gridsearch = SVC(random_state=0)

grid_search_svc = GridSearchCV(svc_gridsearch,parameters,verbose=4,n_jobs=-1,cv=2)

In [11]:
grid_search_svc.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__C', 'estimator__break_ties', 'estimator__cache_size', 'estimator__class_weight', 'estimator__coef0', 'estimator__decision_function_shape', 'estimator__degree', 'estimator__gamma', 'estimator__kernel', 'estimator__max_iter', 'estimator__probability', 'estimator__random_state', 'estimator__shrinking', 'estimator__tol', 'estimator__verbose', 'estimator', 'n_jobs', 'param_grid', 'pre_dispatch', 'refit', 'return_train_score', 'scoring', 'verbose'])

## Entrenamiento de los modelos (sin probar porque no me los termina)

In [12]:
#%time

#grid_search_svc.fit(X_train_norm, y_train) #descomentar


#random_search_polynomic_svc.fit(X_train,y_train)

#### Conseguimos los nuevos hiperparametros

In [13]:
#params_opt_linear_svm = grid_search_svc.best_params_
#params_opt_linear_svm

#obtuve {'C': 1, 'degree': 8, 'kernel': 'rbf', 'max_iter': 50000, 'tol': 1e-07} y lograba converger, con kernel= linear no.

#params_opt_polynomial_svm = random_search_polynomic_svc.best_params_

In [14]:
#linear_svc = SVC().set_params(**params_opt_linear_svm)
#linear_svc = SVC(C=1,tol=1e-3,max_iter=10000)
#polynomial_svc = SVC().set_params(**params_opt_polynomial_svm)

#svc = SVC(C=1,tol=1e-7,kernel= 'rbf',max_iter=50000,random_state=0) #tarda mucho
svc = SVC(C=1,tol=1e-7,kernel= 'poly',max_iter=100000,degree=3,random_state=0)


## Entrenamos de vuelta con svm y los nuevos parametros

In [15]:
svc.fit(X_train_norm,y_train)

#polynomial_svc.fit(X_train,y_train)

/home/bear/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


SVC(C=1, kernel='poly', max_iter=100000, random_state=0, tol=1e-07)

#### Muestra de los resultados

In [16]:
#AGREGAR METODOS DE RECALL, PRECICSION Y FSCORE
y_pred = svc.predict(X_train_norm)
y_pred_test = svc.predict(X_test_norm)



#Usando el entrenado
print("precision = ", precision_score(y_train, y_pred))
print("recall = ", recall_score(y_train, y_pred))
print("auc = ", roc_auc_score(y_train, y_pred))
print("f1 score = ", f1_score(y_train, y_pred))


confusion_matrix(y_train, y_pred)

precision =  0.742463832770378
recall =  0.7478975412467099
auc =  0.8368669845310894
f1 score =  0.7451707816297812


array([[151340,  12123],
       [ 11781,  34950]])

In [17]:
#Usando el test

confusion_matrix(y_test, y_pred_test)


print("precision = ", precision_score(y_test, y_pred_test))
print("recall = ", recall_score(y_test, y_pred_test))
print("auc = ", roc_auc_score(y_test, y_pred_test))
print("f1 score = ", f1_score(y_test, y_pred_test))

precision =  0.6738983623163938
recall =  0.6880710222375452
auc =  0.7968647782201476
f1 score =  0.6809109518935517
